In [ ]:
import numpy as np
import torch
import cv2

from lib.config.project_manager import ProjectManager
from lib.dataset.dataset2D import Dataset2D
from lib.hybridnet.efficienttrack.efficienttrack import EfficientTrack
import matplotlib.pyplot as plt

project = ProjectManager()
project.load('Example_Project')

#project.create_new(
#    name = 'DemodiagStuff',
#    dataset2D_path = '/home/jarvis-setup/Desktop/Testtest',
#    dataset3D_path = '/home/jarvis-setup/Desktop/Testtest')

cfg = project.get_cfg()

In [ ]:
#Load the training and validation sets

training_set = Dataset2D(project.cfg, set='train', mode = 'CenterDetect')
val_set = Dataset2D(project.cfg, set='val',mode = 'CenterDetect')

In [ ]:
#Configuring the network for training

weights = 'pretrained/Monkey_Example/EfficientTrack_Center.pth'
centerDetect = EfficientTrack('CenterDetect', cfg, weights = None)

In [ ]:
#Training the network for 100 epochs

centerDetect.train(training_set, val_set, 100)

In [ ]:
#Configuring the network for inference

weights = "pretrained/Monkey_Example/EfficientTrack_Center.pth"
centerDetect = EfficientTrack('CenterDetectInference', cfg, weights)

In [ ]:
#Load item from validation set and show tracking result
item = val_set.__getitem__(12)
imgs = item[0]
preds, maxvals, img_vis, heatmap = centerDetect.predictCenter(imgs)
plt.imshow(img_vis/255.)
plt.imshow(heatmap, alpha=0.3)
plt.show()